In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.client import device_lib

import numpy as np
import pandas as pd
from collections import defaultdict
import re
import sys
import os



from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model

from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
from matplotlib import pyplot as plt
from keras.layers import LSTM, GRU
from googletrans import Translator
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [1]:
!which python



/home/kab/anaconda3/bin/python


In [4]:
MAX_SEQUENCE_LENGTH = 100
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100


In [171]:
# def clean_str(string):
#     string = re.sub(r"\\", "", string)    
#     string = re.sub(r"\'", "", string)    
#     string = re.sub(r"\"", "", string) 
#     string=re.sub("[^a-zA-Z]+", "",string)
#     return string.strip().lower()

In [172]:
def eng_trans(text):
    t=trans.translate(text,dest="en")
    return t.text

In [200]:
import preprocessor as p

data_train = pd.read_csv('train.csv')



In [201]:
print(data_train.columns.tolist())


['id', 'text', 'label']


In [202]:

# Input Data preprocessing
list_labels = list(set(data_train.label))
print(list_labels)
texts = []
labels = []

for i in range(data_train.text.shape[0]):
    text = p.clean(str(data_train.text[i]))
    texts.append(text)
    labels.append(data_train.label[i])

# for i in range(1000):
#     text = p.clean(str(data_train.text[i]))
#     try:
#         #save text in csv
#         text=eng_trans(text)
#         text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
#         texts.append(text)
#         labels.append(data_train.label[i])
#     except:
#         print("cont")
#         continue

[0, 1, 2]


In [203]:
print(len(texts))

1511


In [204]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 6170 unique tokens.


In [205]:
# Pad input sequences
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(labels),num_classes = len(list_labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# Train test validation Split
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
x_train, x_test, y_train, y_test = train_test_split( data, labels, test_size=0.2
                                                    , random_state=42)
print('Size of train, test:', len(y_train), len(y_test))

print('labels in train, test :')
print(y_train.sum(axis=0))
print(y_test.sum(axis=0))

Shape of data tensor: (1511, 100)
Shape of label tensor: (1511, 3)
Size of train, test: 1208 303
labels in train, test :
[405. 348. 455.]
[ 81.  80. 142.]


In [206]:
word_index = tokenizer.word_index


In [207]:
embeddings_index = {}
f = open(os.path.join("glove.6B", 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [208]:
embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [265]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=data.shape[1],
                            trainable=True)

In [266]:
from keras.regularizers import l2
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=50)


In [267]:
embedding_vecor_length = 32
model = Sequential()
model.add(embedding_layer)


model.add(Conv1D(filters=16, kernel_size=2, padding='valid', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.3,W_regularizer=l2(0.01),return_sequences=True))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.3,W_regularizer=l2(0.01),return_sequences=True))
model.add(LSTM(64,return_sequences=True))
model.add(LSTM(32))
# model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
# model.add(Dense(16, activation='relu'))
model.add(Dense(len(list_labels), activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, dropout=0.2, recurrent_dropout=0.3, return_sequences=True, kernel_regularizer=<keras.reg...)`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, dropout=0.2, recurrent_dropout=0.3, return_sequences=True, kernel_regularizer=<keras.reg...)`
  if __name__ == '__main__':


Model: "sequential_45"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 100, 100)          617100    
_________________________________________________________________
conv1d_42 (Conv1D)           (None, 99, 16)            3216      
_________________________________________________________________
max_pooling1d_42 (MaxPooling (None, 49, 16)            0         
_________________________________________________________________
lstm_72 (LSTM)               (None, 49, 128)           74240     
_________________________________________________________________
lstm_73 (LSTM)               (None, 49, 128)           131584    
_________________________________________________________________
lstm_74 (LSTM)               (None, 49, 64)            49408     
_________________________________________________________________
lstm_75 (LSTM)               (None, 32)              

In [ ]:
model.fit(x_train, y_train, epochs=200, batch_size=64,callbacks=[callback]
         )

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/200
1208/1208 [==============================] - 4s 3ms/step - loss: 2.4321 - accuracy: 0.6667
Epoch 2/200
1208/1208 [==============================] - 3s 2ms/step - loss: 1.5557 - accuracy: 0.6667
Epoch 3/200
1208/1208 [==============================] - 3s 2ms/step - loss: 1.0725 - accuracy: 0.6667
Epoch 4/200
1208/1208 [==============================] - 3s 2ms/step - loss: 0.8239 - accuracy: 0.6680
Epoch 5/200
1208/1208 [==============================] - 3s 2ms/step - loss: 0.7001 - accuracy: 0.6716
Epoch 6/200
1208/1208 [==============================] - 3s 2ms/step - loss: 0.6357 - accuracy: 0.6791
Epoch 7/200
1208/1208 [==============================] - 3s 2ms/step - loss: 0.5927 - accuracy: 0.6912
Epoch 8/200
1208/1208 [==============================] - 3s 2ms/step - loss: 0.5550 - accuracy: 0.7094
Epoch 9/200
1208/1208 [==============================] - 3s 2ms/step - loss: 0.5204 - accuracy: 0.7365
Epoch 10/200
1208/1208 [==============================] - 3s 2ms/step - l

In [261]:
test_preds = model.predict(x_test)
preds = []
actual = []
correct_predictions = 0
for i in range(len(test_preds)):
    x = np.argmax(test_preds[i])
    y = np.argmax(y_test[i])
    preds.append(x)
    actual.append(y)
    if x==y:
        correct_predictions+=1
print("Correct predictions:", correct_predictions)
print("Total number of test examples:", len(y_test))
print("Accuracy of model: ", correct_predictions/float(len(y_test)))

Correct predictions: 153
Total number of test examples: 303
Accuracy of model:  0.504950495049505
